## Introduction to the Term Project

* This project is to provide an opportunity for the students to make use of knowledge learned from this course (computer vision) and practice in real-world problems

* Your project can go to any subfield(s) in computer vision, and we will not make any constraints in the selected topic

* Idea is more important than solution: if you think that time is not sufficient to finish everything, that does not matter – but you need to persuade us that what you have done is promising – of course, a complete project is wonderful

# Automated Deep Learning Design

## Download CIFAR10 Dataset

## Preparation
One time installation of required libraries from requirement.txt and creating data path

In [ ]:
!pip install torch==1.1.0 torchvision==0.2.0
!pip install skorch
!mkdir data

In [ ]:
from modelarts.session import Session
def cifar10_dowloader():
    session = Session()
    bucket_path="/cv-course-public/coding-1/cifar-10-python.tar.gz"
    session.download_data(bucket_path=bucket_path, path="./data/cifar-10-python.tar.gz")
cifar10_dowloader()

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.functional as F
import torchvision
from models.resnet import resnet20, resnet32, resnet56, resnet110
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from skorch import NeuralNetClassifier

## Init train_set with CIFAR10

In [ ]:
train_set = torchvision.datasets.cifar.CIFAR10('data/', train=True, download=True)

## Init ResNet20 model

In [ ]:
model = resnet20()

## Set X & y as train data and train labels

In [ ]:
X = (np.swapaxes(train_set.train_data, 1, 3)/255).astype('float32')
y = np.array(train_set.train_labels).astype('int64')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.01)

## Init CUDA device

In [ ]:
device = torch.device('cuda:0')

## Init NeuralNetClassifier as an interface under raw ResNet20 model with several init hyperparameters

In [ ]:
net = NeuralNetClassifier(
    module=resnet20,
    max_epochs=3,
    lr=0.01,
    optimizer=torch.optim.SGD,
#     optimizer__momentum=0.95,
    device=device
)

In [ ]:
net.fit(X_train, y_train)

## Init hyperparameters grid

In [ ]:
params = {
#     'module': [resnet20, resnet56, resnet110],
#     'lr': [1e-1, 1e-1*2, 1e-1*5, 1e-1*10, 1e-1/2, 1e-1/5, 1e-1/10],
#     'max_epochs': [20, 50, 100],
    'max_epochs': [2, 4, 8, 16],
    'optimizer': [torch.optim.Adam, torch.optim.SGD]
}

## Start Grid Search

For some undefined for now reason Grid Search works not completely right.

So it's still a great example for our research in hyperparameter tuning approaches.

In [ ]:
gs = GridSearchCV(net, params, refit=False, cv=3, scoring='accuracy')
gs.fit(X, y)

## Print output heatmaps

In [ ]:
import matplotlib.pyplot as plt
plt.matshow(gs.cv_results_['mean_test_score'].reshape(-1, 2),
 vmin=0, cmap="viridis")
plt.xlabel("optimizer")
plt.ylabel("max_epochs")
plt.xticks(range(len(params['optimizer'])),
 params['optimizer'])
plt.yticks(range(len(params['max_epochs'])),
 params['max_epochs'])
plt.colorbar()